In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')
np.random.seed(20)
pd.set_option('display.max_rows', None)

Xử lý ngoại lệ

In [2]:
class Gaussion_dist(BaseEstimator, TransformerMixin):
    def __init__(self, feature: str, isInt=True) -> None:
        self.feature = feature
        self.isInt = isInt
    def fit(self, X: pd.DataFrame, y=None):
        self.uppper_boundary = X[self.feature].mean() + 3*X[self.feature].std()
        self.lower_boundary = X[self.feature].mean() - 3*X[self.feature].std()
        if self.isInt:
            self.uppper_boundary = int(self.uppper_boundary)
            self.lower_boundary = int(self.lower_boundary)
        return self
    def transform(self, X: pd.DataFrame):
        X_copy = X.copy()
        X_copy.loc[X_copy[self.feature]>=self.uppper_boundary, self.feature] = self.uppper_boundary
        X_copy.loc[X_copy[self.feature]<=self.lower_boundary, self.feature] = self.lower_boundary
        return X_copy

In [3]:
class Skewed_dist(BaseEstimator, TransformerMixin):
    def __init__(self, feature: str, isInt=True) -> None:
        self.feature = feature
        self.isInt = isInt
    def fit(self, X: pd.DataFrame, y=None):
        IQR = X[self.feature].quantile(0.75) - X[self.feature].quantile(0.25)
        self.uppper_bridge = X[self.feature].quantile(0.75)+(IQR*1.5)
        self.lower_bridge = X[self.feature].quantile(0.25)-(IQR*1.5)
        if self.isInt:
            self.uppper_bridge = int(self.uppper_bridge)
            self.lower_bridge = int(self.lower_bridge)
        return self
    def transform(self, X: pd.DataFrame):
        X_copy = X.copy()
        X_copy.loc[X_copy[self.feature]>=self.uppper_bridge, self.feature] = self.uppper_bridge
        X_copy.loc[X_copy[self.feature]<=self.lower_bridge, self.feature] = self.lower_bridge
        return X_copy

In [4]:
df = pd.read_csv('./clean_data/bigdataset.csv')
df.head()

,Hãng xe,Dòng xe,Năm đăng ký,Số Km đã đi,Tình trạng,Loại xe,Dung tích xe,Xuất xứ,Chính sách bảo hành,Giá
0,Honda,Wave,1996,20000,Đã sử dụng,Xe số,100 - 175 cc,Đang cập nhật,Bảo hành hãng,1500000.0
1,Piaggio,Liberty,2014,3000,Đã sử dụng,Tay ga,100 - 175 cc,Đang cập nhật,Bảo hành hãng,9500000.0
2,Honda,Cub,1992,70000,Đã sử dụng,Xe số,50 - 100 cc,Đang cập nhật,Bảo hành hãng,12000000.0
3,Yamaha,Exciter,2014,29000,Đã sử dụng,Xe số,100 - 175 cc,Đang cập nhật,Bảo hành hãng,9300000.0
4,Yamaha,Nouvo,2011,55555,Đã sử dụng,Tay ga,50 - 100 cc,Đang cập nhật,Bảo hành hãng,6200000.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11499 entries, 0 to 11498
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Hãng xe              11499 non-null  object 
 1   Dòng xe              11499 non-null  object 
 2   Năm đăng ký          11499 non-null  int64  
 3   Số Km đã đi          11499 non-null  int64  
 4   Tình trạng           11499 non-null  object 
 5   Loại xe              11499 non-null  object 
 6   Dung tích xe         11499 non-null  object 
 7   Xuất xứ              11499 non-null  object 
 8   Chính sách bảo hành  11499 non-null  object 
 9   Giá                  11484 non-null  float64
dtypes: float64(1), int64(2), object(7)
memory usage: 898.5+ KB


In [6]:
data = df

In [7]:
data.isna().sum()

Hãng xe                 0
Dòng xe                 0
Năm đăng ký             0
Số Km đã đi             0
Tình trạng              0
Loại xe                 0
Dung tích xe            0
Xuất xứ                 0
Chính sách bảo hành     0
Giá                    15
dtype: int64

In [8]:
data = data.dropna()

In [9]:
data['Loại xe'].unique()

array(['Xe số', 'Tay ga', 'Tay côn/Moto'], dtype=object)

In [10]:
data['Dung tích xe'].unique()

array(['100 - 175 cc', '50 - 100 cc', 'Dưới 50 cc', 'Trên 175 cc',
       'Không biết rõ'], dtype=object)

In [11]:
df_new = pd.get_dummies(data, columns= ['Loại xe', 'Dung tích xe', 'Hãng xe', 'Dòng xe'])

In [12]:
df_new = df_new.drop(['Tình trạng', 'Xuất xứ', 'Chính sách bảo hành'], axis=1)
df_new.head()

,Năm đăng ký,Số Km đã đi,Giá,Loại xe_Tay côn/Moto,Loại xe_Tay ga,Loại xe_Xe số,Dung tích xe_100 - 175 cc,Dung tích xe_50 - 100 cc,Dung tích xe_Dưới 50 cc,Dung tích xe_Không biết rõ,...,Dòng xe_Yass,Dòng xe_Z1000,Dòng xe_Z125,Dòng xe_Z300,Dòng xe_Z650,Dòng xe_Z800,Dòng xe_Z900,Dòng xe_ZX10R,Dòng xe_Zip,Dòng xe_helios
0,1996,20000,1500000.0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2014,3000,9500000.0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1992,70000,12000000.0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2014,29000,9300000.0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2011,55555,6200000.0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
df_new['Năm đăng ký'] = df_new['Năm đăng ký'].astype('int64')
df_new['Giá'] = df_new['Giá'].astype('int64')

In [14]:
df_new = df_new[(df_new['Giá'] < 50000000) & (df_new['Giá'] > 1000000)]
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10006 entries, 0 to 11498
Columns: 250 entries, Năm đăng ký to Dòng xe_helios
dtypes: int64(3), uint8(247)
memory usage: 2.7 MB


In [15]:
X = df_new.drop("Giá", axis=1)
y = df_new["Giá"]

Tìm best feature engineering

In [16]:
# KT xử lý ngoại lệ năm đăng ký
year_outliers = {
    'None': None,
    'Gaussion dist': Gaussion_dist('Năm đăng ký', isInt=True),
    'Skewed dist': Skewed_dist('Năm đăng ký', isInt=True),
}

# KT xử lý ngoại lệ số km đã đi
number_of_km_outliers = {
    'None': None,
    'Gaussion dist': Gaussion_dist('Số Km đã đi', isInt=False),
    'Skewed dist': Skewed_dist('Số Km đã đi', isInt=False),
}

from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler, RobustScaler, Normalizer, QuantileTransformer, PowerTransformer
# KT chuẩn hóa
transformers = {
    "None": None,
    "MinMaxScaler": MinMaxScaler(),
    "MaxAbsScaler": MaxAbsScaler(),
    "StandardScaler": StandardScaler(),
    "RobustScaler": RobustScaler(),
    "Normalizer": Normalizer(),
    "QuantileTransformer": QuantileTransformer(n_quantiles=100, output_distribution='normal'),
    "PowerTransformer": PowerTransformer(),
}

In [17]:
def r2_score_predict(X: pd.DataFrame, y: pd.DataFrame, pipe: Pipeline):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=32)
    pipe.fit(X_train, y_train)
    r2_train = r2_score(y_train, pipe.predict(X_train))
    r2_test = r2_score(y_test, pipe.predict(X_test))
    return r2_train, r2_test

In [18]:
def getPipeline(year_outlier=None,number_of_km_outlier=None, transformer=None, function=None):
    steps = []
    if year_outlier != None:
        steps.append(('year_outlier', year_outlier))
    if number_of_km_outlier != None:
        steps.append(('number_of_km_outlier', number_of_km_outlier))
    if transformer != None:
        steps.append(('transformer', transformer))
    steps.append(('PCA', PCA(n_components=30)))
    steps.append(('LinearRegression',function))
    return Pipeline(steps=steps)

In [21]:
def get_result(X,y):
    result = {
        "KT xử lý ngoại lệ năm đăng ký": [],
        "KT xử lý ngoại lệ số km đã đi": [],
        "KT chuẩn hóa": [],
        "Độ chính xác theo R2_Score train": [],
        "Độ chính xác theo R2_Score test": []
    }
    for k1, year_outlier in year_outliers.items():
        for k2, number_of_km_outlier in number_of_km_outliers.items():
            for k3, transformer in transformers.items():
                pipe = getPipeline(year_outlier=year_outlier,  number_of_km_outlier=number_of_km_outlier, transformer=transformer, function=GradientBoostingRegressor())
                accuracy1_train, accuracy1_test = r2_score_predict(X, y, pipe)
                result["KT xử lý ngoại lệ năm đăng ký"].append(k1)
                result["KT xử lý ngoại lệ số km đã đi"].append(k2)
                result["KT chuẩn hóa"].append(k3)
                result["Độ chính xác theo R2_Score train"].append(accuracy1_train)
                result["Độ chính xác theo R2_Score test"].append(accuracy1_test)
    return result

In [20]:
result = get_result(X,y)

In [ ]:
result = pd.DataFrame(result).sort_values(by=["Độ chính xác theo R2_Score test"], ascending=False).reset_index(drop=True)
result

,KT xử lý ngoại lệ năm đăng ký,KT xử lý ngoại lệ số km đã đi,KT chuẩn hóa,Độ chính xác theo R2_Score train,Độ chính xác theo R2_Score test
0,Skewed dist,Skewed dist,None,0.679594,0.609338
1,Gaussion dist,Skewed dist,None,0.686286,0.609082
2,Skewed dist,None,None,0.681749,0.606316
3,Gaussion dist,None,MinMaxScaler,0.671979,0.605072
4,Gaussion dist,None,None,0.678959,0.605016
5,Skewed dist,Gaussion dist,None,0.678014,0.603763
6,Gaussion dist,Gaussion dist,None,0.678667,0.603637
7,None,Gaussion dist,None,0.682886,0.603315
8,None,None,MinMaxScaler,0.673584,0.602514
9,Skewed dist,None,MinMaxScaler,0.675638,0.601809


In [ ]:
result.iloc[result['Độ chính xác theo R2_Score test'].idxmax()]

KT xử lý ngoại lệ năm đăng ký       Skewed dist
KT xử lý ngoại lệ số km đã đi       Skewed dist
KT chuẩn hóa                               None
Độ chính xác theo R2_Score train       0.679594
Độ chính xác theo R2_Score test        0.609338
Name: 0, dtype: object

In [ ]:
df = pd.read_csv('./clean_data/smalldataset.csv')
data = df
data.isna().sum()
df_new = pd.get_dummies(data, columns= ['Loại xe', 'Dung tích xe', 'Hãng xe', 'Dòng xe'])
df_new = df_new.drop(['Tình trạng', 'Xuất xứ', 'Chính sách bảo hành'], axis=1)
df_new = df_new[(df_new['Giá'] < 50000000) & (df_new['Giá'] > 1000000)]
X_small = df_new.drop("Giá", axis=1)
y_small = df_new["Giá"]

In [ ]:
result = get_result(X_small,y_small)
result = pd.DataFrame(result).sort_values(by=["Độ chính xác theo R2_Score test"], ascending=False).reset_index(drop=True)
result.iloc[result['Độ chính xác theo R2_Score test'].idxmax()]

KT xử lý ngoại lệ năm đăng ký                 None
KT xử lý ngoại lệ số km đã đi        Gaussion dist
KT chuẩn hóa                        StandardScaler
Độ chính xác theo R2_Score train          0.683113
Độ chính xác theo R2_Score test           0.570711
Name: 0, dtype: object

Tìm tham số tối ưu

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=32) 
feature_engineering_pipe = make_pipeline(Skewed_dist('Năm đăng ký', isInt=True), Skewed_dist('Số Km đã đi', isInt=False), PCA(n_components=30))
X_train = feature_engineering_pipe.fit_transform(X_train)
X_test = feature_engineering_pipe.transform(X_test)
GBR = GradientBoostingRegressor()
parameters = {
            'learning_rate': [0.01,0.02,0.03,0.04],
            'subsample'    : [0.9, 0.5, 0.2, 0.1],
            'n_estimators' : [100,500,1000, 1500],
            'max_depth'    : [4,6,8,10]
            }
grid_GBR = GridSearchCV(estimator=GBR, param_grid = parameters, cv = 5, n_jobs=-1)a
grid_GBR.fit(X_train, y_train)
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",grid_GBR.best_estimator_)
print("\n The best score across ALL searched params:\n",grid_GBR.best_score_)
print("\n The best parameters across ALL searched params:\n",grid_GBR.best_params_)

KeyboardInterrupt: 

In [20]:
grid_GBR.score(X_test, y_test)

0.6933693964153498